In [69]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

sys.path.append("..")
from otc.metrics.metrics import effective_spread

import wandb

from tqdm.auto import tqdm

from scipy.stats import wilcoxon
import warnings


In [2]:
# set here globally
exchange = "cboe"
models = ["gbm", "classical"]
subset = "test"  # "all"
strategy = "transfer"  # "supervised"


In [3]:
key = f"{exchange}_all_{strategy}_{subset}"
dataset = f"fbv/thesis/{exchange}_{strategy}_raw:latest"

os.environ["GCLOUD_PROJECT"] = "flowing-mantis-239216"

run = wandb.init(project="thesis", entity="fbv")

# load unscaled data
artifact = run.use_artifact(dataset)  # type: ignore
data_dir = artifact.download()

# load results
result_dirs = []
for model in models:
    results = f"fbv/thesis/{exchange}_{model}_{strategy}_{subset}:latest"
    artifact = run.use_artifact(results)  # type: ignore
    result_dir = artifact.download()
    result_dirs.append(result_dir)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: karelze (fbv). Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact cboe_transfer_raw:latest, 663.29MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.0
wandb: Downloading large artifact cboe_gbm_transfer_test:latest, 67.57MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.0
wandb: Downloading large artifact cboe_classical_transfer_test:latest, 365.80MB. 1 files... 
/pfs/data5/home/kit/stud/uloak/thesis/thesis/lib/python3.8/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
wandb:   1 of 1 files downloaded.  
Done. 0:0:8.2


In [30]:
# p. 35-38
columns = [
    "buy_sell",
    "EXPIRATION",
    "QUOTE_DATETIME",
    "TRADE_SIZE",
    "TRADE_PRICE",
    "ask_ex",
    "bid_ex",
    "myn",
    "OPTION_TYPE",
    "issue_type",
]


if subset == "all":
    train = pd.read_parquet(
        Path(data_dir, "train_set"), engine="fastparquet", columns=columns
    )
    val = pd.read_parquet(
        Path(data_dir, "val_set"), engine="fastparquet", columns=columns
    )
    test = pd.read_parquet(
        Path(data_dir, "test_set"), engine="fastparquet", columns=columns
    )
    eval_data = pd.concat([train, val, test])
    del train, val, test

elif subset == "test":
    eval_data = pd.read_parquet(
        Path(data_dir, "test_set"), engine="fastparquet", columns=columns
    )


results = []
for i, model in tqdm(enumerate(models)):
    result = pd.read_parquet(Path(result_dirs[i], "results"), engine="fastparquet")
    result.columns = pd.MultiIndex.from_product([[model], result.columns])
    results.append(result)

results_data = pd.concat(results, axis=1, names=models)

assert len(eval_data) == len(results_data)

X_print = eval_data

del results


2it [00:02,  1.28s/it]


In [31]:
results_data


gbm classical                                        \
         gbm(classical-size) tick(all) tick(ex) quote(best) quote(ex) lr(ex)   
index                                                                          
24700819                  -1      -1.0      1.0        -1.0      -1.0   -1.0   
24700261                  -1      -1.0      1.0        -1.0      -1.0   -1.0   
24700842                  -1       1.0      1.0        -1.0      -1.0    1.0   
24695039                   1      -1.0     -1.0         1.0      -1.0   -1.0   
24700841                   1       1.0      1.0         1.0       1.0    1.0   
...                      ...       ...      ...         ...       ...    ...   
37155409                   1      -1.0      1.0        -1.0      -1.0   -1.0   
37155410                   1       1.0      1.0         1.0       1.0    1.0   
37104659                  -1      -1.0     -1.0         1.0       1.0    1.0   
37108365                   1      -1.0     -1.0        -1.0      -1.0   -1.0   
37155411                   1      -1.0     -1.0        -1.0      -1.0   -1.0   

                                                   ...                         \
         lr(best) rev_lr(ex) rev_lr(best) emo(ex)  ... rev_emo(best) clnv(ex)   
index                                              ...                          
24700819     -1.0        1.0         -1.0    -1.0  ...           1.0     -1.0   
24700261     -1.0       -1.0         -1.0    -1.0  ...          -1.0     -1.0   
24700842     -1.0       -1.0         -1.0     1.0  ...          -1.0      1.0   
24695039      1.0       -1.0          1.0    -1.0  ...           1.0     -1.0   
24700841      1.0        1.0          1.0     1.0  ...           1.0      1.0   
...           ...        ...          ...     ...  ...           ...      ...   
37155409     -1.0       -1.0         -1.0    -1.0  ...          -1.0     -1.0   
37155410      1.0        1.0          1.0     1.0  ...          -1.0      1.0   
37104659      1.0        1.0          1.0     1.0  ...           1.0      1.0   
37108365     -1.0       -1.0         -1.0    -1.0  ...          -1.0     -1.0   
37155411     -1.0       -1.0         -1.0    -1.0  ...          -1.0     -1.0   

                                                                           \
         clnv(best) rev_clnv(ex) rev_clnv(best) trade_size(ex)->tick(all)   
index                                                                       
24700819       -1.0          1.0            1.0                      -1.0   
24700261       -1.0         -1.0           -1.0                      -1.0   
24700842       -1.0         -1.0           -1.0                       1.0   
24695039        1.0         -1.0            1.0                      -1.0   
24700841        1.0          1.0            1.0                       1.0   
...             ...          ...            ...                       ...   
37155409       -1.0         -1.0           -1.0                      -1.0   
37155410        1.0         -1.0           -1.0                       1.0   
37104659        1.0          1.0            1.0                      -1.0   
37108365       -1.0         -1.0           -1.0                       1.0   
37155411       -1.0         -1.0           -1.0                       1.0   

                                                                             \
         trade_size(ex)->quote(best) trade_size(ex)->quote(best)->quote(ex)   
index                                                                         
24700819                        -1.0                                   -1.0   
24700261                        -1.0                                   -1.0   
24700842                        -1.0                                   -1.0   
24695039                         1.0                                    1.0   
24700841                         1.0                                    1.0   
...                              ...                      

### Robustness Checks

In [32]:
# prepare columns for printing
X_print["ttm"] = (
    X_print["EXPIRATION"].dt.to_period("M")
    - X_print["QUOTE_DATETIME"].dt.to_period("M")
).apply(lambda x: x.n)

X_print["year"] = X_print["QUOTE_DATETIME"].dt.year

bins_tradesize = [-1, 1, 3, 5, 11, np.inf]
trade_size_labels = ["(0,1]", "(1,3]", "(3,5]", "(5,11]", ">11"]
X_print["TRADE_SIZE_binned"] = pd.cut(
    X_print["TRADE_SIZE"], bins_tradesize, labels=trade_size_labels
)

# p. 38
bins_years = [2004, 2007, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]
year_labels = [
    "2005-2007",
    "2008-2010",
    "2011",
    "2012",
    "2013",
    "2014",
    "2015",
    "2016",
    "2017",
]
X_print["year_binned"] = pd.cut(X_print["year"], bins_years, labels=year_labels)

# p. 37
bins_ttm = [-1, 1, 2, 3, 6, 12, np.inf]
ttm_labels = [
    "ttm <= 1 month",
    "ttm (1-2] month",
    "ttm (2-3] month",
    "ttm (3-6] month",
    "ttm (6-12] month",
    "ttm > 12 month",
]
X_print["ttm_binned"] = pd.cut(X_print["ttm"], bins_ttm, labels=ttm_labels)

# Security type
# see 3.0a-mb-explanatory-data-analysis.ipynb
X_print["issue_type"] = X_print["issue_type"].map(
    {
        "0": "Stock options",
        "A": "Index option",
        "7": "Others",
        "F": "Others",
        "%": "Others",
        " ": "Others",
    }
)

# Moneyness p. 38
bins_myn = [-1, 0.7, 0.9, 1.1, 1.3, np.inf]
myn_labels = [
    "myn <= 0.7",
    "myn (0.7-0.9]",
    "myn (0.9-1.1]",
    "myn (1.1-1.3]",
    "myn > 1.3",
]
X_print["myn_binned"] = pd.cut(X_print["myn"], bins_myn, labels=myn_labels)

# mid
mid_ex = 0.5 * (X_print["ask_ex"] + X_print["bid_ex"])
X_print["mid"] = mid_ex

# p. 31
def map_quotes(x):
    if x["TRADE_PRICE"] == x["bid_ex"] or x["TRADE_PRICE"] == x["ask_ex"]:
        return "at quote"
    elif x["TRADE_PRICE"] < x["bid_ex"] or x["TRADE_PRICE"] > x["ask_ex"]:
        return "outside"
    elif x["TRADE_PRICE"] == x["mid"]:
        return "at mid"
    # TODO: See my mail to C Grauer. How to handle unknowns?
    elif (
        x["bid_ex"] < x["TRADE_PRICE"] < x["mid"]
        or x["mid"] < x["TRADE_PRICE"] < x["ask_ex"]
    ):
        return "inside"
    return "other"


X_print["prox_q_binned"] = X_print[["TRADE_PRICE", "mid", "bid_ex", "ask_ex"]].apply(
    map_quotes, axis=1
)

# clean up empty buckets, as it causes empty grouping in result set generatio
X_print["year_binned"] = X_print["year_binned"].cat.remove_unused_categories()
X_print["myn_binned"] = X_print["myn_binned"].cat.remove_unused_categories()
X_print["ttm_binned"] = X_print["ttm_binned"].cat.remove_unused_categories()


X_print.drop(
    columns=[
        "EXPIRATION",
        "QUOTE_DATETIME",
        "TRADE_SIZE",
        "ttm",
        "myn",
        "ask_ex",
        "bid_ex",
        "year",
    ],
    inplace=True,
)


In [33]:
X_print.head(20)


,buy_sell,TRADE_PRICE,OPTION_TYPE,issue_type,TRADE_SIZE_binned,year_binned,ttm_binned,myn_binned,mid,prox_q_binned
index,,,,,,,,,,
24700819,-1,1.730000,C,Others,"(0,1]",2015,ttm (1-2] month,myn (0.9-1.1],NaN,other
24700261,-1,0.600000,C,Stock options,"(1,3]",2015,ttm <= 1 month,myn (0.9-1.1],0.80,at quote
24700842,-1,1.300000,C,Others,"(1,3]",2015,ttm <= 1 month,myn (0.9-1.1],NaN,other
24695039,1,1.280000,C,Others,"(0,1]",2015,ttm <= 1 month,myn (0.9-1.1],NaN,other
24700841,-1,1.040000,C,Others,>11,2015,ttm <= 1 month,myn (0.9-1.1],1.00,at quote
182536,-1,3.300000,P,Others,"(0,1]",2015,ttm (3-6] month,myn (0.7-0.9],NaN,other
24700164,1,0.250000,C,Stock options,>11,2015,ttm <= 1 month,myn (0.9-1.1],NaN,other
24323781,1,64.239998,P,Index option,"(0,1]",2015,ttm (3-6] month,myn (0.9-1.1],NaN,other
24700851,1,8.950000,C,Stock options,"(0,1]",2015,ttm <= 1 month,myn (0.9-1.1],8.50,inside


In [34]:
X_print = pd.concat([X_print, results_data], axis=1)


In [35]:
X_print.head()


,buy_sell,TRADE_PRICE,OPTION_TYPE,issue_type,TRADE_SIZE_binned,year_binned,ttm_binned,myn_binned,mid,prox_q_binned,...,"(classical, rev_emo(best))","(classical, clnv(ex))","(classical, clnv(best))","(classical, rev_clnv(ex))","(classical, rev_clnv(best))","(classical, trade_size(ex)->tick(all))","(classical, trade_size(ex)->quote(best))","(classical, trade_size(ex)->quote(best)->quote(ex))","(classical, quote(best)->quote(ex))","(classical, trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(ex))"
index,,,,,,,,,,,,,,,,,,,,,
24700819,-1,1.73,C,Others,"(0,1]",2015,ttm (1-2] month,myn (0.9-1.1],NaN,other,...,1.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0
24700261,-1,0.60,C,Stock options,"(1,3]",2015,ttm <= 1 month,myn (0.9-1.1],0.8,at quote,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
24700842,-1,1.30,C,Others,"(1,3]",2015,ttm <= 1 month,myn (0.9-1.1],NaN,other,...,-1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0
24695039,1,1.28,C,Others,"(0,1]",2015,ttm <= 1 month,myn (0.9-1.1],NaN,other,...,1.0,-1.0,1.0,-1.0,1.0,-1.0,1.0,1.0,1.0,1.0
24700841,-1,1.04,C,Others,>11,2015,ttm <= 1 month,myn (0.9-1.1],1.0,at quote,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


## Results Set Generation

In [36]:
LUT = {
    "(ex)": " (ex)",
    "(best)": " (best)",
    "rev_": "Rev. ",
    "tick": "Tick",
    "quote": "Quote",
    "trade_size": "Trade Size",
    "depth": "Depth",
    "->": " $\\to$ ",
    "lr": "\gls{LR}",
    "emo": "\gls{EMO}",
    "clnv": "\gls{CLNV}",
    "OPTION_TYPE": "Option Type",
    "(": "$(",  # put interval start in math env
    "]": "]$",  # put interval end in math env
    "_": "$\_",
}

LUT_INDEX = {
    "OPTION_TYPE": "Option Type",
    "issue_type": "Security Type",
    "TRADE_SIZE_binned": "Trade Size",
    "year_binned": "Year",
    "ttm_binned": "Time to Maturity",
    "myn_binned": "Moneyness",
    "prox_q_binned": "Location to Quote",
}


def cell_str(x):
    for orig, sub in LUT.items():
        x = x.replace(orig, sub)
    return x


def highlight_max(s, props=""):
    return np.where(s == np.nanmax(s.values), props, "")


In [37]:
def set_tex_style(styler, caption, label):
    res = (
        styler.set_caption(caption)
        # .hide(axis="index")
        .apply(
            highlight_max, props="font-weight:bold;", axis=0
        )  # optionally set subset....
        .format_index(cell_str, axis=0)
        .format_index(cell_str, axis=1)
        .format(precision=4, decimal=".", thousands=",", escape=False, hyperlinks=None)
        .to_latex(
            f"{label}.tex",
            siunitx=True,
            position_float="centering",
            hrules=True,
            clines="skip-last;data",
            label="tab:" + label,
            caption=caption,
            convert_css=True,
        )
    )
    return res


In [38]:
classifiers = results_data.columns.tolist()
criterions = list(LUT_INDEX)


In [48]:
results_data.columns.tolist()


[('gbm', 'gbm(classical-size)'),
 ('classical', 'tick(all)'),
 ('classical', 'tick(ex)'),
 ('classical', 'quote(best)'),
 ('classical', 'quote(ex)'),
 ('classical', 'lr(ex)'),
 ('classical', 'lr(best)'),
 ('classical', 'rev_lr(ex)'),
 ('classical', 'rev_lr(best)'),
 ('classical', 'emo(ex)'),
 ('classical', 'emo(best)'),
 ('classical', 'rev_emo(ex)'),
 ('classical', 'rev_emo(best)'),
 ('classical', 'clnv(ex)'),
 ('classical', 'clnv(best)'),
 ('classical', 'rev_clnv(ex)'),
 ('classical', 'rev_clnv(best)'),
 ('classical', 'trade_size(ex)->tick(all)'),
 ('classical', 'trade_size(ex)->quote(best)'),
 ('classical', 'trade_size(ex)->quote(best)->quote(ex)'),
 ('classical', 'quote(best)->quote(ex)'),
 ('classical',
  'trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(ex)')]

## Accuracy Calculation

In [55]:
X_print.columns


Index([                                                                                   'buy_sell',
                                                                                       'TRADE_PRICE',
                                                                                       'OPTION_TYPE',
                                                                                        'issue_type',
                                                                                 'TRADE_SIZE_binned',
                                                                                       'year_binned',
                                                                                        'ttm_binned',
                                                                                        'myn_binned',
                                                                                               'mid',
                                                                                  

In [70]:
# FIXME: Find better approach
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

result_dfs = []

for criterion in tqdm(criterions):
    results = []
    for classifier in tqdm(classifiers):
        acc_tot = accuracy_score(
            X_print["buy_sell"].astype("int8"), X_print[classifier]
        )

        res = (
            X_print.groupby([criterion])[["buy_sell", classifier]]
            .apply(
                lambda x: accuracy_score(x["buy_sell"].astype("int8"), x[classifier])
            )
            .mul(100)
            .rename(classifier)
        )
        res.loc["all"] = acc_tot * 100

        res.index.name = LUT_INDEX.get(criterion)
        results.append(res)

    # save aggregated results
    result_df = pd.concat(results, axis=1).T
    result_df.style.pipe(
        set_tex_style,
        caption=(f"long-tbd", "short-tbd"),
        label=f"{key.lower()}-{criterion.lower()}",
    )

    # store all result sets for later use
    result_dfs.append(result_df)


100%|██████████| 7/7 [09:08<00:00, 78.40s/it]


In [71]:
master = pd.concat(result_dfs, axis=1, keys=list(LUT_INDEX.values())).T


In [72]:
master


gbm  classical             \
                                   gbm(classical-size)  tick(all)   tick(ex)   
Option Type       C                          71.726305  50.068165  48.639452   
                  P                          72.263779  50.885849  49.446618   
                  all                        71.970823  50.440161  49.006663   
Security Type     Index option               67.418516  48.293046  48.174157   
                  Others                     74.140903  50.500341  48.745063   
                  Stock options              71.521690  50.628366  49.197934   
                  all                        71.970823  50.440161  49.006663   
Trade Size        (0,1]                      70.805402  49.300227  48.806380   
                  (1,3]                      71.384306  49.681809  48.943375   
                  (3,5]                      71.967362  50.049784  48.731932   
                  (5,11]                     71.571455  50.425808  48.827326   
                  >11                        74.498617  53.030815  49.696146   
                  all                        71.970823  50.440161  49.006663   
Year              2015                       71.497521  50.346261  48.779877   
                  2016                       71.624954  50.420550  48.904024   
                  2017                       72.398824  50.472687  49.144306   
                  all                        71.970823  50.440161  49.006663   
Time to Maturity  ttm <= 1 month             72.015004  50.679230  48.975387   
                  ttm (1-2] month            72.648398  51.301982  49.791415   
                  ttm (2-3] month            72.321337  50.112909  48.990408   
                  ttm (3-6] month            72.895032  49.515227  48.819425   
                  ttm (6-12] month           71.950704  49.498436  48.591253   
                  ttm > 12 month             66.808042  49.318407  48.433594   
                  all                        71.970823  50.440161  49.006663   
Moneyness         myn <= 0.7                 73.013351  51.738718  49.728784   
                  myn (0.7-0.9]              72.201759  50.549440  48.423386   
                  myn (0.9-1.1]              72.631488  50.575395  49.278613   
                  myn (1.1-1.3]              66.175326  48.645103  47.879624   
                  myn > 1.3                  64.543886  48.063069  47.501014   
                  all                        71.970823  50.440161  49.006663   
Location to Quote at mid                     67.930315  45.393812  45.332425   
                  at quote                   74.793421  51.099545  48.553174   
                  inside                     71.992303  50.997505  49.583989   
                  other                      83.383004  50.925675  49.869051   
                  outside                    71.324540  56.815059  53.050289   
                  all                        71.970823  50.440161  49.006663   

                                                                      \
                                   quote(best)  quote(ex)     lr(ex)   
Option Type       C                  59.216378  61.855015  61.529399   
                  P                  60.486601  62.281154  62.011341   
                  all                59.794251  62.048882  61.748653   
Security Type     Index option       53.302115  53.865554  53.648287   
                  Others             63.366657  66.064760  65.669597   
                  Stock options      58.955429  61.193841  60.924802   
                  all                59.794251  62.048882  61.748653   
Trade Size        (0,1]              56.129130  58.840355  58.626470   
                  (1,3]              58.705402  62.018777  61.732442   
                  (3,5]              59.453704  62.431819  62.139861   
                  (5,11]             60.378448  62.782615  62.449727   
                  >11                65.475073  65.364517  64.960174   
  

In [74]:
master.style.pipe(
    set_tex_style, caption=("master-short", "master-long"), label=f"{key}-master"
)


## Effective Spread 🚧

In [ ]:
classifiers


In [ ]:
eff_dfs = []


def stats(x, classifier):

    nom = effective_spread(x[classifier], x["TRADE_PRICE"], x["mid"], mode="nominal")
    rel = (
        effective_spread(x[classifier], x["TRADE_PRICE"], x["mid"], mode="relative")
        * 100
    )

    # eff_spread_pred = effective_spread(x[classifier], x["TRADE_PRICE"], x["mid"], mode="none")
    # eff_spread_true = effective_spread(x["buy_sell"], x["TRADE_PRICE"], x["mid"], mode="none")
    # wilcoxon_res  = wilcoxon(eff_spread_pred, eff_spread_true, nan_policy="omit", zero_method="zsplit")

    return pd.Series(
        {
            "nominal": nom,
            "rel": rel,
            # 'statistic':wilcoxon_res.statistic,
            # 'pvalue':wilcoxon_res.pvalue
        }
    )


for criterion in tqdm(criterions):
    results = []

    for classifier in tqdm(classifiers):
        res = X_print.groupby([criterion])[
            ["TRADE_PRICE", "mid", classifier, "buy_sell"]
        ].apply(stats, classifier)
        results.append(res)

    # save aggregated results
    result_df = pd.concat(results, axis=1, keys=classifiers).T
    result_df.style.pipe(
        set_tex_style,
        caption=(f"long-tbd", "short-tbd"),
        label=f"{key.lower()}-{criterion.lower()}-eff-spread",
    )

    # store all result sets for later use
    eff_dfs.append(result_df)


In [ ]:
master = pd.concat(eff_dfs, axis=1, keys=LUT_INDEX.values()).T


In [ ]:
master


In [ ]:
master.iloc[:, 0:20]


In [ ]:
master.iloc[:, 21:40]


In [ ]:
master.iloc[:, 41:-1]


In [ ]:
master.style.pipe(
    set_tex_style,
    caption=("master-short", "master-long"),
    label=f"{key}-master-eff-spread",
)


## Change in Parenthesis 🅾️

```latex
# https://tex.stackexchange.com/questions/430283/table-with-numbers-in-parentheses-in-siunitx/430290#430290
\begin{table}
    \centering
    \caption{test of combination with change}
    \label{tab:combo}
    \begin{tabular}{lSSSSSSSS}
        \toprule
        {} & \multicolumn{2}{l}{Index option} & \multicolumn{2}{l}{Others} & \multicolumn{2}{l}{Stock options} & \multicolumn{2}{l}{all} \\
        \midrule
        classical-size & 1.0 & \parl-56.42\parr & 2.0 & \parl-74.35\parr & -73.5 & \parl-143.93\parr & 5.0 & \parl-67.33\parr \\
        \bottomrule
        \end{tabular}
\end{table}
```

In [ ]:
foo = pd.DataFrame([[4, 3, 8, 5]])
bar = pd.DataFrame([[1.1, 2, 73, 5]], columns=foo.columns, index=foo.index)


In [ ]:
def combine_results(revised: pd.DataFrame, base: pd.DataFrame) -> pd.DataFrame:
    """
    Generate print layout like in Grauer et al.

    https://tex.stackexchange.com/questions/430283/table-with-numbers-in-parentheses-in-siunitx/430290#430290

    # see p. https://texdoc.org/serve/siunitx/0
    """
    # first, second layer of colum index
    c_1 = revised.columns
    c_2 = ["nom"]
    midx = pd.MultiIndex.from_product([c_1, c_2])

    # copy data from revised add as (column, "nom")
    combo = pd.DataFrame(revised.values, index=revised.index, columns=midx)

    for i, mul_col in enumerate(combo.columns):

        # define custom brackets that are not parsed by sunitx
        combo[[(mul_col[0], "pm")]] = (
            "\parl" + (combo[mul_col] - base[mul_col[0]]).round(2).astype(str) + "\parr"
        )
        # sort to group together columns
        combo.sort_index(axis=1, inplace=True)
    return combo


In [ ]:
combo = combine_results(bar, foo)

# manually replace S with S[table-format=1.4(5)] if needed
combo.style.to_latex(
    f"combo.tex",
    siunitx=True,
    position_float="centering",
    hrules=True,
    clines="skip-last;data",
    label="tab:combo",
    caption="test of combination with change",
    multicol_align="l",
)


In [ ]:
combo
